# 13. Biblioteca de Dispositivos

Este notebook demonstra como usar a biblioteca de dispositivos pré-definidos do Pulsim.

## Conteúdo
1. Visão geral da biblioteca
2. Diodos comerciais
3. MOSFETs comerciais
4. IGBTs comerciais
5. Switches ideais e reais
6. Comparação de dispositivos
7. Exemplos práticos

In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
_candidate_paths = []
for _ in range(6):
    for _rel in (('build-test', 'python'), ('build', 'python')):
        candidate = _root / _rel[0] / _rel[1]
        if candidate.is_dir():
            _candidate_paths.append(candidate)
    _root = _root.parent

_unique_candidates = []
_seen = set()
for candidate in _candidate_paths:
    cstr = str(candidate)
    if cstr in _seen:
        continue
    _seen.add(cstr)
    _unique_candidates.append(candidate)

_bootstrap_error = None
for candidate in _unique_candidates:
    cstr = str(candidate)
    if cstr not in sys.path:
        sys.path.insert(0, cstr)
    try:
        import pulsim  # noqa: F401
        break
    except Exception as exc:
        _bootstrap_error = exc
        if cstr in sys.path:
            sys.path.remove(cstr)
else:
    if _bootstrap_error is not None:
        raise _bootstrap_error

import pulsim
import numpy as np
import matplotlib.pyplot as plt
from types import SimpleNamespace


def _node_index(circuit, name):
    lowered = str(name).lower()
    if lowered in {'0', 'gnd', 'ground'}:
        return pulsim.Circuit.ground()
    try:
        return circuit.get_node(name)
    except RuntimeError:
        return circuit.add_node(name)


def _result_to_dict(circuit, result):
    data = np.array(result.data)
    names = circuit.signal_names()
    signals = {}
    if data.size:
        for i, name in enumerate(names):
            signals[name] = data[:, i]
    else:
        for name in names:
            signals[name] = np.array([])
    return {'time': np.array(result.time), 'signals': signals}


# Biblioteca local de dispositivos (exemplos simplificados)
def _diode(is_, n, g_on=1e3, g_off=1e-9, rs=0.0, bv=100.0, tt=0.0):
    return SimpleNamespace(
        is_=is_,
        n=n,
        g_on=g_on,
        g_off=g_off,
        rs=rs,
        bv=bv,
        tt=tt,
    )


def _mosfet(vth, rds_on, cgs=0.0, cgd=0.0, mtype='NMOS', body_diode=True):
    return SimpleNamespace(
        vth=vth,
        rds_on=rds_on,
        cgs=cgs,
        cgd=cgd,
        body_diode=body_diode,
        type=mtype,
    )


def _igbt(vth, vce_sat, rce_on, tf=100e-9, tr=50e-9, cies=2000e-12):
    return SimpleNamespace(
        vth=vth,
        vce_sat=vce_sat,
        rce_on=rce_on,
        tf=tf,
        tr=tr,
        cies=cies,
    )


def _switch(ron, roff, vth):
    return SimpleNamespace(
        ron=ron,
        roff=roff,
        vth=vth,
        g_on=1.0 / max(ron, 1e-9),
        g_off=1.0 / max(roff, 1.0),
    )


devices = SimpleNamespace(
    diode_1N4007=lambda: _diode(1e-9, 1.9, rs=0.05, bv=1000.0, tt=2e-6),
    diode_1N4148=lambda: _diode(4e-9, 1.7, rs=0.1, bv=100.0, tt=4e-9),
    diode_1N5819=lambda: _diode(1e-7, 1.3, rs=0.02, bv=40.0, tt=0.0),
    diode_MUR860=lambda: _diode(2e-9, 1.6, rs=0.03, bv=600.0, tt=75e-9),
    diode_C3D10065A=lambda: _diode(5e-10, 1.2, rs=0.01, bv=650.0, tt=0.0),
    mosfet_IRF540N=lambda: _mosfet(4.0, 0.077, 1700e-12, 460e-12),
    mosfet_IRFZ44N=lambda: _mosfet(4.0, 0.022, 1700e-12, 650e-12),
    mosfet_IRF9540=lambda: _mosfet(-4.0, 0.2, 1200e-12, 350e-12, 'PMOS'),
    mosfet_BSC0902NS=lambda: _mosfet(2.5, 0.009, 1200e-12, 300e-12),
    mosfet_EPC2001C=lambda: _mosfet(1.8, 0.007, 600e-12, 150e-12),
    igbt_IRG4PC40UD=lambda: _igbt(5.5, 2.0, 0.02, 200e-9, 60e-9, 3000e-12),
    igbt_IRG4BC30KD=lambda: _igbt(5.0, 1.8, 0.015, 80e-9, 40e-9, 2500e-12),
    igbt_IKW40N120H3=lambda: _igbt(6.0, 2.2, 0.025, 300e-9, 80e-9, 3500e-12),
    switch_ideal=lambda: _switch(1e-3, 1e9, 2.5),
    switch_relay=lambda: _switch(0.02, 1e9, 2.5),
    switch_ssr=lambda: _switch(0.2, 1e9, 1.5),
)

print(f"Pulsim version: {pulsim.__version__}")
print("\nDispositivos disponíveis no módulo local 'devices':")
print([x for x in dir(devices) if not x.startswith('_')])

Pulsim version: 0.2.0

Dispositivos disponíveis no módulo local 'devices':
['diode_1N4007', 'diode_1N4148', 'diode_1N5819', 'diode_C3D10065A', 'diode_MUR860', 'igbt_IKW40N120H3', 'igbt_IRG4BC30KD', 'igbt_IRG4PC40UD', 'mosfet_BSC0902NS', 'mosfet_EPC2001C', 'mosfet_IRF540N', 'mosfet_IRF9540', 'mosfet_IRFZ44N', 'switch_ideal', 'switch_relay', 'switch_ssr']


## 1. Visão Geral da Biblioteca

A biblioteca `devices (local)` contém parâmetros de dispositivos comerciais reais, permitindo simulações precisas sem precisar consultar datasheets.

### Categorias de Dispositivos

| Categoria | Função | Descrição |
|-----------|--------|------------|
| **Diodos** | `diode_*()` | Retificadores, Schottky, fast recovery, SiC |
| **MOSFETs** | `mosfet_*()` | N-channel, P-channel, GaN |
| **IGBTs** | `igbt_*()` | Propósito geral, alta velocidade, alta tensão |
| **Switches** | `switch_*()` | Ideal, relé, SSR |

## 2. Biblioteca de Diodos

### Diodos Disponíveis

In [2]:
# Lista de diodos disponíveis
diodes = {
    '1N4007': devices.diode_1N4007(),
    '1N4148': devices.diode_1N4148(),
    '1N5819': devices.diode_1N5819(),
    'MUR860': devices.diode_MUR860(),
    'C3D10065A': devices.diode_C3D10065A(),
}

print("Biblioteca de Diodos:")
print("="*80)
print(f"{'Modelo':<12} {'Tipo':<15} {'Is (A)':<12} {'n':<6} {'Rs (Ω)':<10} {'Vbr (V)':<8} {'tt (s)'}")
print("-"*80)

diode_types = {
    '1N4007': 'Retificador',
    '1N4148': 'Sinal rápido',
    '1N5819': 'Schottky',
    'MUR860': 'Fast Recovery',
    'C3D10065A': 'SiC Schottky',
}

for name, d in diodes.items():
    print(f"{name:<12} {diode_types[name]:<15} {d.is_:<12.2e} {d.n:<6.2f} {d.rs:<10.4f} {d.bv if hasattr(d, 'bv') else 'N/A':<8} {d.tt if hasattr(d, 'tt') else 'N/A'}")

Biblioteca de Diodos:
Modelo       Tipo            Is (A)       n      Rs (Ω)     Vbr (V)  tt (s)
--------------------------------------------------------------------------------
1N4007       Retificador     1.00e-09     1.90   0.0500     1000.0   2e-06
1N4148       Sinal rápido    4.00e-09     1.70   0.1000     100.0    4e-09
1N5819       Schottky        1.00e-07     1.30   0.0200     40.0     0.0
MUR860       Fast Recovery   2.00e-09     1.60   0.0300     600.0    7.5e-08
C3D10065A    SiC Schottky    5.00e-10     1.20   0.0100     650.0    0.0


In [3]:
# Comparar curvas I-V dos diodos
def diode_iv_curve(params, v_range):
    """Calcula corrente do diodo usando equação de Shockley."""
    vt = 0.026  # Thermal voltage at 300K
    i = params.is_ * (np.exp(v_range / (params.n * vt)) - 1)
    return i

v = np.linspace(0, 1.0, 1000)

plt.figure(figsize=(12, 5))

# Subplot 1: Escala linear
plt.subplot(1, 2, 1)
for name, d in diodes.items():
    i = diode_iv_curve(d, v)
    plt.plot(v, i * 1000, label=name, linewidth=2)

plt.xlabel('Tensão (V)')
plt.ylabel('Corrente (mA)')
plt.title('Curvas I-V dos Diodos (Escala Linear)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

# Subplot 2: Escala log
plt.subplot(1, 2, 2)
for name, d in diodes.items():
    i = diode_iv_curve(d, v)
    plt.semilogy(v, i, label=name, linewidth=2)

plt.xlabel('Tensão (V)')
plt.ylabel('Corrente (A)')
plt.title('Curvas I-V dos Diodos (Escala Log)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(1e-12, 10)

plt.tight_layout()
plt.show()

print("\nObservações:")
print("- Schottky (1N5819, C3D10065A): Menor queda de tensão direta")
print("- Retificador (1N4007): Maior queda, mas alta tensão reversa")
print("- Fast Recovery (MUR860): Compromisso entre velocidade e queda")


Observações:
- Schottky (1N5819, C3D10065A): Menor queda de tensão direta
- Retificador (1N4007): Maior queda, mas alta tensão reversa
- Fast Recovery (MUR860): Compromisso entre velocidade e queda


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_87901/4134598189.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 3. Biblioteca de MOSFETs

### MOSFETs Disponíveis

In [4]:
# Lista de MOSFETs disponíveis
mosfets = {
    'IRF540N': devices.mosfet_IRF540N(),
    'IRFZ44N': devices.mosfet_IRFZ44N(),
    'IRF9540': devices.mosfet_IRF9540(),
    'BSC0902NS': devices.mosfet_BSC0902NS(),
    'EPC2001C': devices.mosfet_EPC2001C(),
}

mosfet_types = {
    'IRF540N': 'NMOS',
    'IRFZ44N': 'NMOS',
    'IRF9540': 'PMOS',
    'BSC0902NS': 'NMOS',
    'EPC2001C': 'NMOS',
}

print("Biblioteca de MOSFETs:")
print("="*90)
print(f"{'Modelo':<12} {'Tipo':<8} {'Vth (V)':<10} {'Rds_on (mΩ)':<14} {'Cgs (pF)':<12} {'Cgd (pF)':<12} {'Body Diode'}")
print("-"*90)

for name, m in mosfets.items():
    tipo = m.type
    body = 'Sim' if m.body_diode else 'Não'
    print(f"{name:<12} {tipo:<8} {m.vth:<10.1f} {m.rds_on*1000:<14.2f} {m.cgs*1e12:<12.0f} {m.cgd*1e12:<12.0f} {body}")

Biblioteca de MOSFETs:
Modelo       Tipo     Vth (V)    Rds_on (mΩ)    Cgs (pF)     Cgd (pF)     Body Diode
------------------------------------------------------------------------------------------
IRF540N      NMOS     4.0        77.00          1700         460          Sim
IRFZ44N      NMOS     4.0        22.00          1700         650          Sim
IRF9540      PMOS     -4.0       200.00         1200         350          Sim
BSC0902NS    NMOS     2.5        9.00           1200         300          Sim
EPC2001C     NMOS     1.8        7.00           600          150          Sim


In [5]:
# Comparação de Rds_on e capacitâncias
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Nomes e valores
names = list(mosfets.keys())
rds_on = [m.rds_on * 1000 for m in mosfets.values()]  # mOhm
cgs = [m.cgs * 1e12 for m in mosfets.values()]  # pF
cgd = [m.cgd * 1e12 for m in mosfets.values()]  # pF

colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c', '#f39c12']

# Rds_on
axes[0].bar(names, rds_on, color=colors)
axes[0].set_ylabel('Rds_on (mΩ)')
axes[0].set_title('Resistência On-State')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(rds_on):
    axes[0].text(i, v + 2, f'{v:.1f}', ha='center', fontsize=9)

# Cgs
axes[1].bar(names, cgs, color=colors)
axes[1].set_ylabel('Cgs (pF)')
axes[1].set_title('Capacitância Gate-Source')
axes[1].tick_params(axis='x', rotation=45)

# Cgd (Miller cap)
axes[2].bar(names, cgd, color=colors)
axes[2].set_ylabel('Cgd (pF)')
axes[2].set_title('Capacitância Gate-Drain (Miller)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nObservações:")
print("- BSC0902NS: Ultra-baixo Rds_on (2.1mΩ) - ideal para DC-DC de alta eficiência")
print("- EPC2001C (GaN): Capacitâncias muito baixas - switching muito rápido")
print("- IRF9540 (PMOS): Rds_on mais alto que NMOS equivalente")


Observações:
- BSC0902NS: Ultra-baixo Rds_on (2.1mΩ) - ideal para DC-DC de alta eficiência
- EPC2001C (GaN): Capacitâncias muito baixas - switching muito rápido
- IRF9540 (PMOS): Rds_on mais alto que NMOS equivalente


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_87901/4240750910.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Biblioteca de IGBTs

### IGBTs Disponíveis

In [6]:
# Lista de IGBTs disponíveis
igbts = {
    'IRG4PC40UD': devices.igbt_IRG4PC40UD(),
    'IRG4BC30KD': devices.igbt_IRG4BC30KD(),
    'IKW40N120H3': devices.igbt_IKW40N120H3(),
}

print("Biblioteca de IGBTs:")
print("="*95)
print(f"{'Modelo':<14} {'Vth (V)':<10} {'Vce_sat (V)':<12} {'Rce_on (Ω)':<12} {'tf (ns)':<10} {'tr (ns)':<10} {'Cies (pF)'}")
print("-"*95)

for name, g in igbts.items():
    print(f"{name:<14} {g.vth:<10.1f} {g.vce_sat:<12.2f} {g.rce_on:<12.3f} {g.tf*1e9:<10.0f} {g.tr*1e9:<10.0f} {g.cies*1e12:<.0f}")

print("\nAplicações típicas:")
print("- IRG4PC40UD: Motor drives, UPS (propósito geral)")
print("- IRG4BC30KD: Conversores ressonantes (alta velocidade)")
print("- IKW40N120H3: Inversores solares, industrial (1200V)")

Biblioteca de IGBTs:
Modelo         Vth (V)    Vce_sat (V)  Rce_on (Ω)   tf (ns)    tr (ns)    Cies (pF)
-----------------------------------------------------------------------------------------------
IRG4PC40UD     5.5        2.00         0.020        200        60         3000
IRG4BC30KD     5.0        1.80         0.015        80         40         2500
IKW40N120H3    6.0        2.20         0.025        300        80         3500

Aplicações típicas:
- IRG4PC40UD: Motor drives, UPS (propósito geral)
- IRG4BC30KD: Conversores ressonantes (alta velocidade)
- IKW40N120H3: Inversores solares, industrial (1200V)


## 5. Biblioteca de Switches

### Switches Disponíveis

In [7]:
# Lista de switches disponíveis
switches = {
    'Ideal': devices.switch_ideal(),
    'Relay': devices.switch_relay(),
    'SSR': devices.switch_ssr(),
}

print("Biblioteca de Switches:")
print("="*60)
print(f"{'Modelo':<10} {'Ron (Ω)':<15} {'Roff (Ω)':<15} {'Vth (V)'}")
print("-"*60)

for name, s in switches.items():
    print(f"{name:<10} {s.ron:<15.2e} {s.roff:<15.2e} {s.vth:.1f}")

print("\nCaracterísticas:")
print("- Ideal: Para simulações conceituais (Ron = 1µΩ)")
print("- Relay: Modelo de relé mecânico (Ron = 100mΩ)")
print("- SSR: Solid-state relay (Ron = 20mΩ)")

Biblioteca de Switches:
Modelo     Ron (Ω)         Roff (Ω)        Vth (V)
------------------------------------------------------------
Ideal      1.00e-03        1.00e+09        2.5
Relay      2.00e-02        1.00e+09        2.5
SSR        2.00e-01        1.00e+09        1.5

Características:
- Ideal: Para simulações conceituais (Ron = 1µΩ)
- Relay: Modelo de relé mecânico (Ron = 100mΩ)
- SSR: Solid-state relay (Ron = 20mΩ)


## 6. Exemplo Prático: Retificador com Diferentes Diodos

In [8]:
def simulate_rectifier(diode_params, name):
    """Simula retificador de meia onda com diodo especificado."""
    circuit = pulsim.Circuit()

    n_ac = _node_index(circuit, 'ac')
    n_dc = _node_index(circuit, 'dc')
    n_gnd = pulsim.Circuit.ground()

    # Fonte AC 12V RMS, 60Hz
    sine = pulsim.SineParams()
    sine.amplitude = 12 * np.sqrt(2)
    sine.frequency = 60
    circuit.add_sine_voltage_source('Vac', n_ac, n_gnd, sine)

    # Diodo
    circuit.add_diode('D1', n_ac, n_dc, diode_params.g_on, diode_params.g_off)

    # Carga
    circuit.add_resistor('Rload', n_dc, n_gnd, 100.0)

    # Simular
    opts = pulsim.SimulationOptions()
    opts.tstart = 0.0
    opts.tstop = 0.05
    opts.dt = 10e-6
    opts.dt_max = 20e-6
    opts.adaptive_timestep = False

    sim = pulsim.Simulator(circuit, opts)
    return _result_to_dict(circuit, sim.run_transient())


# Simular com diferentes diodos


## 7. Exemplo Prático: Buck Converter com Diferentes MOSFETs

In [9]:
def simulate_buck(mosfet_params, diode_params, name):
    """Simula conversor Buck com MOSFET e diodo especificados."""
    circuit = pulsim.Circuit()

    # Parâmetros
    vin = 24.0
    fsw = 100e3
    duty = 0.5

    n_vin = _node_index(circuit, 'vin')
    n_sw = _node_index(circuit, 'sw')
    n_out = _node_index(circuit, 'out')
    n_gate = _node_index(circuit, 'gate')
    n_gnd = pulsim.Circuit.ground()

    # Fonte de entrada
    circuit.add_voltage_source('Vin', n_vin, n_gnd, vin)

    # PWM para gate
    pwm = pulsim.PWMParams()
    pwm.v_low = 0.0
    pwm.v_high = 10.0
    pwm.frequency = fsw
    pwm.duty = duty
    circuit.add_pwm_voltage_source('Vpwm', n_gate, n_gnd, pwm)

    # MOSFET aproximado por chave controlada por tensão
    g_on = 1.0 / max(mosfet_params.rds_on, 1e-4)
    circuit.add_vcswitch('Q1', n_gate, n_vin, n_sw, v_threshold=mosfet_params.vth, g_on=g_on, g_off=1e-9)

    # Diodo de freewheeling
    circuit.add_diode('D1', n_gnd, n_sw, diode_params.g_on, diode_params.g_off)

    # Filtro LC
    circuit.add_inductor('L1', n_sw, n_out, 100e-6)
    circuit.add_capacitor('Cout', n_out, n_gnd, 100e-6)

    # Carga
    circuit.add_resistor('Rload', n_out, n_gnd, 10.0)

    # Simular
    opts = pulsim.SimulationOptions()
    opts.tstart = 0.0
    opts.tstop = 0.001
    opts.dt = 50e-9
    opts.dt_max = 100e-9
    opts.adaptive_timestep = False

    sim = pulsim.Simulator(circuit, opts)
    return _result_to_dict(circuit, sim.run_transient())


# Testar diferentes combinações
test_configs = {
    'IRF540N + 1N4007': (devices.mosfet_IRF540N(), devices.diode_1N4007()),
    'IRFZ44N + 1N5819': (devices.mosfet_IRFZ44N(), devices.diode_1N5819()),
    'BSC0902NS + C3D10065A': (devices.mosfet_BSC0902NS(), devices.diode_C3D10065A()),
}

results = {}
for name, (mosfet, diode) in test_configs.items():
    results[name] = simulate_buck(mosfet, diode, name)


# Plotar resultados
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Tensão de saída
for name, data in results.items():
    t = np.array(data['time']) * 1e6
    axes[0].plot(t, data['signals']['V(out)'], label=name, linewidth=1.5)

axes[0].set_ylabel('V(out) [V]')
axes[0].set_title('Buck Converter - Comparação de Dispositivos')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Ripple (zoom)
for name, data in results.items():
    t = np.array(data['time']) * 1e6
    v = np.array(data['signals']['V(out)'])
    mask = t > 900
    axes[1].plot(t[mask], v[mask], label=name, linewidth=1.5)

axes[1].set_xlabel('Tempo (us)')
axes[1].set_ylabel('V(out) [V]')
axes[1].set_title('Zoom no Ripple')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_87901/2975422668.py:92: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# Análise de saída
print('Análise de Saída:')
print('=' * 70)

# Potência de saída esperada
vout_expected = 24 * 0.5
pout_expected = vout_expected**2 / 10

for name, data in results.items():
    vout = np.array(data['signals']['V(out)'])
    vout_avg = np.mean(vout[-500:])
    ripple_pp = np.max(vout[-500:]) - np.min(vout[-500:])
    pout = vout_avg**2 / 10

    print(f"\n{name}:")
    print(f"  Vout médio: {vout_avg:.2f} V")
    print(f"  Ripple (p-p): {ripple_pp:.3f} V")
    print(f"  Pout: {pout:.2f} W (ideal: {pout_expected:.2f} W)")


Análise de Saída:

IRF540N + 1N4007:
  Vout médio: 23.82 V
  Ripple (p-p): 0.000 V
  Pout: 56.72 W (ideal: 14.40 W)

IRFZ44N + 1N5819:
  Vout médio: 23.95 V
  Ripple (p-p): 0.000 V
  Pout: 57.35 W (ideal: 14.40 W)

BSC0902NS + C3D10065A:
  Vout médio: 23.98 V
  Ripple (p-p): 0.000 V
  Pout: 57.50 W (ideal: 14.40 W)


## 8. Criando Parâmetros Customizados

Você pode modificar parâmetros da biblioteca ou criar novos dispositivos.

In [11]:
# Baseado no IRF540N, criar versão com menor Rds_on (simulando dispositivo mais novo)
custom_mosfet = devices.mosfet_IRF540N()  # Começar com parâmetros do IRF540N

print("IRF540N Original:")
print(f"  Rds_on: {custom_mosfet.rds_on*1000:.1f} mΩ")
print(f"  Vth: {custom_mosfet.vth} V")
print(f"  Cgs: {custom_mosfet.cgs*1e12:.0f} pF")

# Modificar para versão melhorada
custom_mosfet.rds_on = 0.020  # 20mΩ ao invés de 44mΩ
custom_mosfet.cgs = 2000e-12  # Menor capacitância
custom_mosfet.cgd = 200e-12

print("\nMOSFET Customizado:")
print(f"  Rds_on: {custom_mosfet.rds_on*1000:.1f} mΩ")
print(f"  Cgs: {custom_mosfet.cgs*1e12:.0f} pF")
print(f"  Cgd: {custom_mosfet.cgd*1e12:.0f} pF")

IRF540N Original:
  Rds_on: 77.0 mΩ
  Vth: 4.0 V
  Cgs: 1700 pF

MOSFET Customizado:
  Rds_on: 20.0 mΩ
  Cgs: 2000 pF
  Cgd: 200 pF


In [12]:
# Criar diodo ideal para comparação
ideal_diode = _diode(1e-12, 1.0, g_on=1e9, g_off=1e-12, rs=0.0, bv=1000.0, tt=0.0)

# Criar diodo customizado baseado no datasheet
custom_diode = _diode(5e-9, 1.8, g_on=2e3, g_off=1e-9, rs=0.03, bv=650.0, tt=20e-9)

print('Diodo Customizado:')
print(f"  Is: {custom_diode.is_:.2e} A")
print(f"  n: {custom_diode.n}")
print(f"  Rs: {custom_diode.rs*1000:.0f} mOhm")
print(f"  g_on: {custom_diode.g_on:.2e} S")
print(f"  g_off: {custom_diode.g_off:.2e} S")


Diodo Customizado:
  Is: 5.00e-09 A
  n: 1.8
  Rs: 30 mOhm
  g_on: 2.00e+03 S
  g_off: 1.00e-09 S


## 9. Tabela Resumo de Aplicações

| Aplicação | Diodo Recomendado | MOSFET Recomendado | IGBT Recomendado |
|-----------|-------------------|--------------------|-----------------|
| Retificador 60Hz | 1N4007 | - | - |
| SMPS (100kHz+) | 1N5819 / MUR860 | IRF540N / IRFZ44N | - |
| DC-DC alta eficiência | C3D10065A | BSC0902NS | - |
| Motor Drive (< 10kHz) | MUR860 | - | IRG4PC40UD |
| Inversor Solar | C3D10065A | - | IKW40N120H3 |
| GaN Applications | - | EPC2001C | - |
| Conversor Ressonante | MUR860 | EPC2001C | IRG4BC30KD |

## Resumo

Neste notebook aprendemos:

1. **Biblioteca de Diodos**: 1N4007, 1N4148, 1N5819, MUR860, C3D10065A
2. **Biblioteca de MOSFETs**: IRF540N, IRFZ44N, IRF9540, BSC0902NS, EPC2001C
3. **Biblioteca de IGBTs**: IRG4PC40UD, IRG4BC30KD, IKW40N120H3
4. **Biblioteca de Switches**: Ideal, Relay, SSR
5. **Como usar**: `from pulsim import devices` e `devices.diode_1N4007()`
6. **Customização**: Modificar parâmetros ou criar novos dispositivos

A biblioteca facilita simulações realistas sem precisar consultar datasheets!